# Use Spark for R to load data and run SQL queries
This notebook introduces basic Spark concepts and helps you to start using Spark for R.

Some familiarity with R is recommended. This notebook runs on R with Spark 2.0.

In this notebook, you'll use the publicly available **mtcars** data set from *Motor Trend* magazine to learn some basic R. You'll learn how to load data, create a Spark DataFrame, aggregate data, run mathematical formulas, and run SQL queries against the data.

## Table of contents
This notebook contains these main sections:

1. [Load a DataFrame](#Load_a_DataFrame)
2. [Initialize an SQLContext](#Initialize_an_SQLContext)
3. [Create a Spark DataFrame](#Create_a_Spark_DataFrame)
4. [Aggregate data after grouping by columns](#Aggregate_data_after_grouping_by_columns)
5. [Operate on columns](#Operate_on_columns)
6. [Run SQL queries from the Spark DataFrame](#Run_SQL_queries_from_the_Spark_DataFrame)

<a id='Load_a_DataFrame'></a>
## 1. Load a DataFrame
A DataFrame is a distributed collection of data that is organized into named columns. The built-in R DataFrame called **mtcars** includes observations on the following 11 variables:

`[, 1]	mpg     Miles / (US) gallon`  
`[, 2]	cyl     Number of cylinders`  
`[, 3]	disp	Displacement (cu. in.)`  
`[, 4]	hp      Gross horsepower`  
`[, 5]	drat    Rear axle ratio`  
`[, 6]	wt      Weight (1000 lbs)`  
`[, 7]	qsec    1/4 mile time (seconds)`  
`[, 8]	vs      0 = V-engine, 1 = straight engine`  
`[, 9]	am      Transmission (0 = automatic, 1 = manual)`  
`[,10]	gear    Number of forward gears`  
`[,11]	carb    Number of carburetors`

Preview the first 3 rows of the DataFrame by using the head() function:

In [1]:
#DEPENDS
# Since this display_data would be transformed into mark down of <table></table>
head(mtcars, 3)

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1


Convert the car name data, which appears in the row names, into an actual column so that Spark can read it as a column:

In [2]:
#DEPENDS
mtcars$car <- rownames(mtcars)
mtcars <- mtcars[,c(12,1:11)]
rownames(mtcars) <- 1:nrow(mtcars)
head(mtcars)

car,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360,175,3.15,3.440,17.02,0,0,3,2
Valiant,18.1,6,225,105,2.76,3.460,20.22,1,0,3,1


<a id='Initialize_an_SQLContext'></a>
## 2. Initialize an SQLContext
To work with a DataFrame, you need an SQLContext. You create this SQLContext by using `sparkRSQL.init(sc)`. A SparkContext named sc, which has been created for you, is used to initialize the SQLContext:

In [3]:
sqlContext <- sparkR.session(sc)

<a id='Create_a_Spark_DataFrame'></a>
## 3. Create a Spark DataFrame
Using the SQLContext and the loaded local DataFrame, create a Spark DataFrame and print the schema, or structure, of the DataFrame:

In [4]:
sdf <- createDataFrame(mtcars, schema = NULL) 
printSchema(sdf)

root
 |-- car: string (nullable = true)
 |-- mpg: double (nullable = true)
 |-- cyl: double (nullable = true)
 |-- disp: double (nullable = true)
 |-- hp: double (nullable = true)
 |-- drat: double (nullable = true)
 |-- wt: double (nullable = true)
 |-- qsec: double (nullable = true)
 |-- vs: double (nullable = true)
 |-- am: double (nullable = true)
 |-- gear: double (nullable = true)
 |-- carb: double (nullable = true)


Display the content of the Spark DataFrame:

In [5]:
#DEPENDS
SparkR::head(sdf, 32)

car,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2
Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1
Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4
Merc 240D,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2
Merc 230,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2
Merc 280,19.2,6,167.6,123,3.92,3.440,18.30,1,0,4,4


Try different ways of retrieving subsets of the data. For example, get the first 5 values in the **mpg** column:

In [6]:
#DEPENDS
SparkR::head(select(sdf, sdf$mpg),5)

mpg
21.0
21.0
22.8
21.4
18.7


Filter the DataFrame to retain only rows with **mpg** values that are less than 18:

In [7]:
#DEPENDS
SparkR::head(SparkR::filter(sdf, sdf$mpg < 18))

car,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Duster 360,14.3,8,360.0,245,3.21,3.57,15.84,0,0,3,4
Merc 280C,17.8,6,167.6,123,3.92,3.44,18.90,1,0,4,4
Merc 450SE,16.4,8,275.8,180,3.07,4.07,17.40,0,0,3,3
Merc 450SL,17.3,8,275.8,180,3.07,3.73,17.60,0,0,3,3
Merc 450SLC,15.2,8,275.8,180,3.07,3.78,18.00,0,0,3,3
Cadillac Fleetwood,10.4,8,472.0,205,2.93,5.25,17.98,0,0,3,4


<a id='Aggregate_data_after_grouping_by_columns'></a>
## 4. Aggregate data after grouping by columns
Spark DataFrames support a number of common functions to aggregate data after grouping. For example, you can compute the average weight of cars as a function of the number of cylinders:

In [8]:
#DEPENDS
SparkR::head(summarize(groupBy(sdf, sdf$cyl), wtavg = avg(sdf$wt)))

cyl,wtavg
8,3.999214
4,2.285727
6,3.117143


You can also sort the output from the aggregation to determine the most popular cylinder configuration in the DataFrame:

In [9]:
#DEPENDS
car_counts <-summarize(groupBy(sdf, sdf$cyl), count = n(sdf$wt))
SparkR::head(arrange(car_counts, desc(car_counts$count)))

cyl,count
8,14
4,11
6,7


<a id='Operate_on_columns'></a>
## 5. Operate on columns
SparkR provides a number of functions that you can apply directly to columns for data processing. In the following example, a basic arithmetic function converts lbs to metric tons:

In [10]:
#DEPENDS
sdf$wtTon <- sdf$wt * 0.45
SparkR::head(select(sdf, sdf$car, sdf$wt, sdf$wtTon),6)

car,wt,wtTon
Mazda RX4,2.620,1.17900
Mazda RX4 Wag,2.875,1.29375
Datsun 710,2.320,1.04400
Hornet 4 Drive,3.215,1.44675
Hornet Sportabout,3.440,1.54800
Valiant,3.460,1.55700


<a id='Run_SQL_queries_from_the_Spark_DataFrame'></a>
## 6. Run SQL queries from the Spark DataFrame
You can register a Spark DataFrame as a temporary table and then run SQL queries over the data. The `sql` function enables an application to run SQL queries programmatically and returns the result as a DataFrame:

In [11]:
#DEPENDS
registerTempTable(sdf, "cars")

highgearcars <- sql(sqlContext, "SELECT car, gear FROM cars WHERE gear >= 5")
SparkR::head(highgearcars)

Warning message:
“'registerTempTable' is deprecated.
Use 'createOrReplaceTempView' instead.
See help("Deprecated")”Warning message:
“'sql(sqlContext...)' is deprecated.
Use 'sql(sqlQuery)' instead.
See help("Deprecated")”

car,gear
Porsche 914-2,5
Lotus Europa,5
Ford Pantera L,5
Ferrari Dino,5
Maserati Bora,5


## That's it!
You successfully completed this notebook! You learned how to load a DataFrame, view and filter the data, aggregate the data, perform operations on the data in specific columns, and run SQL queries against the data. For more information about Spark, see the [Spark Quick Start Guide](http://spark.apache.org/docs/latest/quick-start.html).

## Want to learn more?
### Free courses on <a href="https://bigdatauniversity.com/courses/?utm_source=tutorial-dashdb-python&utm_medium=github&utm_campaign=bdu/" rel="noopener noreferrer" target="_blank">Big Data University</a>: <a href="https://bigdatauniversity.com/courses/?utm_source=tutorial-dashdb-python&utm_medium=github&utm_campaign=bdu" rel="noopener noreferrer" target="_blank"><img src = "https://ibm.box.com/shared/static/xomeu7dacwufkoawbg3owc8wzuezltn6.png" width=600px> </a>

### Authors

**Saeed Aghabozorgi**, PhD, is a Data Scientist in IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge. He is a researcher in the data mining field and an expert in developing advanced analytic methods like machine learning and statistical modelling on large data sets.

**Polong Lin** is a Data Scientist at IBM in Canada. Under the Emerging Technologies division, Polong is responsible for educating the next generation of data scientists through Big Data University. Polong is a regular speaker in conferences and meetups, and holds an M.Sc. in Cognitive Psychology.

Copyright © 2016, 2017 Big Data University. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/" rel="noopener noreferrer" target="_blank">MIT License</a>.